# 一致预期 price 表

# R03

In [1]:

class MysqlConn(object):
    def __init__(self, config_json_str):
        # self.mysql_config = {"host": "10.22.128.150",  "port": 3317, "db": "bigdata", "user": "talend_load",
        #   "passwd": "s9t5gNThn2vqWM7c" , "charset" : "utf8"}
        # self.mysql_config = {"host": "db-bigdata.wmcloud-qa.com",  "port": 3312, "db": "bigdata", "user": "app_bigdata_ro",
        #   "passwd": "Welcome_20141217"}
        self.mysql_config = json.loads(config_json_str, encoding='utf-8')

    def connect(self):
        mysql_conn = pymysql.connect(**self.mysql_config)
        return mysql_conn

class MssqlConn(object):
    def __init__(self, config_json_str):
        # self.mssql_config = {"server": "sh-datayesdb.wmcloud-dev.com",  "port": 1433, "database": "datayesdb",
        #     "user": "talend_load", "password": "Welcome01"}
        self.mssql_config = json.loads(config_json_str, encoding='utf-8')

    def connect(self):
        mssql_conn = pymssql.connect(**self.mssql_config)
        return mssql_conn

    def __str__(self):
        return json.dumps(self.mssql_config, ensure_ascii=False, encoding='utf-8')



In [2]:
import json
import pymysql
import pymssql
# MySQL
# bigdata={"host":"security03-dev.datayes.com","port":3306,"user":"talend_load","passwd":"NCph1G9BQT3DuQj","db":"bigdata","charset":"utf8"}
# dateyesdbp={"host":"db-datayesdb-ro.wmcloud.com","port":3313,"user":"app_dataqa_ro","passwd":"Welcome20140820","db":"datayesdbp","charset":"utf8"}
# datayesdb={"host":"db-datayesdb-ro.wmcloud.com","port":3313,"database":"datayesdb","user":"app_gaea_ro","password":"EQw6WquhnCKPp8Li","charset":"utf8"}

# researchrpt={"host":"db-researchrpt-ro.wmcloud.com","port":3306,"user":"app_gaea_ro","passwd":"UDVVkvULzakP2048","db":"research_rpt","charset":"utf8"}
# dev
# researchrpt={"host":"10.24.21.34","port":3306,"user":"app_researchrep","passwd":"dG8pqlk5VKOMvDH","db":"research_rpt","charset":"utf8"}
# prd
researchrpt={"host":"db-researchrpt.wmcloud.com","port":3306,"user":"app_researchrep","passwd":"dG8pqlk5VKOMvDH","db":"research_rpt","charset":"utf8"}
bigdata={"host":"real-stg01.datayes.com","port":3314,"user":"app_talend_rw","passwd":"KbcwQ32gamVR3sPw","db":"bigdata","charset":"utf8"}

# SQLserver
# datayes={"server":"sh-dm-db05.datayes.com","port":1433,"database":"dyedb","user":"talend_load","password":"s9t5gNThn2vqWM7c","charset":"utf8"}
datayesdb={"server":"10.21.139.72","port":1433,"database":"datayesdb","user":"talend_load","password":"s9t5gNThn2vqWM7c","charset":"utf8"}



# bigdata_conn = MysqlConn(json.dumps(bigdata)).connect()
# dateyesdbp_conn = MysqlConn(json.dumps(dateyesdbp)).connect()
# datayesdb_conn = MysqlConn(json.dumps(datayesdb)).connect()
researchrpt_conn = MysqlConn(json.dumps(researchrpt)).connect()

datayesdb_conn = MssqlConn(json.dumps(datayesdb)).connect()
# datayes_conn = MssqlConn(json.dumps(datayes)).connect()


In [3]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [4]:
# base_df
filter_str = "CREATE_DATE" # 根据 num 列分开1，2 后， 需要从num=1的数据中筛选出日期为查询日期的数据
this_date_str = '2019-01-30'
primary_key = ["CODE", "ORGAN_ID"] # 用于merge
last_key = ["CREATE_DATE", "TEXT5"] # 同一个机构上次对本证券做的预测报告的撰写日期 同一个机构上次对本证券做的预测报告的评级

In [16]:
sql = """
SELECT ORIGIN_ID, CODE, CODE_NAME, TITLE, TEXT5, AUTHOR, CREATE_DATE, ORGAN_ID, num
FROM (
	SELECT ORIGIN_ID, CODE, CODE_NAME, TITLE, TEXT5, AUTHOR, CREATE_DATE, ORGAN_ID,
		   (@row_number:=
			   case when CODE = @pre_code and ORGAN_ID = @pre_organ_id
			then @row_number+1
			 else 1
			end
		   ) as num,
		(@pre_code := CODE),
		(@pre_organ_id := ORGAN_ID)
	FROM cmb_report_research,(select @row_number:=1,@pre_code:='',@pre_organ_id:='') a
	where CREATE_DATE <= '%s 00:00:00'
	order by CODE,ORGAN_ID,CREATE_DATE desc
) a
where a.num<=2
""" % (this_date_str)
base_df = pd.read_sql(sql, researchrpt_conn)

In [10]:
base_df.head()
# base_df["TEXT5"].values
# base_df = base_df.dropna(subset=["TEXT5"])

In [17]:
# 用num的值区分次序
base_df["num"] = base_df["num"].astype(int)
last_df = base_df[base_df["num"]==2]
last_df = last_df.drop("num", 1)

In [18]:
last_df.head()

,ORIGIN_ID,CODE,CODE_NAME,TITLE,TEXT5,AUTHOR,CREATE_DATE,ORGAN_ID
1,3523516,,金蝶国际,基建投资需求紧迫，轨交信息化产业迎来拐点,NaN,"杨墨,李傲远",2019-01-13,2
3,3509742,,中联重科-A,12月挖掘机销量点评：2018年销量创新高，龙头份额显著提升,NaN,"孔令鑫,张梓丁",2019-01-07,3
5,3602834,,安踏体育,纺织服装行业1月第四周周报：板块维持低配，出口年末下滑,NaN,"黄淑妍,张家璇",2019-01-21,10
7,3509155,,呷哺呷哺,每周回顾：元旦景区客流疲软，口碑3月起收取平台费,NaN,"姜娅,武明戈,杨清朴",2019-01-06,13
10,3642985,,志邦股份,轻工行业周报：2018年轻工行业重要数据回顾；Q4基金持仓整体下降,NaN,徐林锋,2019-01-27,38


In [19]:
base_df[filter_str] = base_df[filter_str].astype(str)

this_df = base_df[(base_df["num"]==1) & (base_df[filter_str]==this_date_str)]
this_df = this_df.drop("num", 1)


In [20]:
if len(this_df) == 0:
    # 说明没有目标数据， 可以返回
    pass


In [21]:
this_df.head()

,ORIGIN_ID,CODE,CODE_NAME,TITLE,TEXT5,AUTHOR,CREATE_DATE,ORGAN_ID
0,3644585,,中广核电力,2018年电力数据点评：2018年用电量同比增长8.5%，火电利用小时显著提升,NaN,"周妍,张爱宁",2019-01-30,2
42,3646379,000001,平安银行,Q4银行持仓微降，仓位反弹可期,NaN,"沈娟,郭其伟",2019-01-30,77
54,3645684,000001,平安银行,2月策略：政策与资金驱动，反弹行情或延续,NaN,廖志明,2019-01-30,2570
58,3645685,000002,万科A,对当前板块表现的讨论（十二）：楼市的小阳春,NaN,"谢皓宇,卜文凯,白淑媛",2019-01-30,2
105,3645207,000002,万科A,房地产销售周报：基本面依旧较弱、看好1月下旬后地产股机会,NaN,"陈天诚,徐超",2019-01-30,2570


In [22]:
key = last_key + primary_key


In [23]:
key

['CREATE_DATE', 'TEXT5', 'CODE', 'ORGAN_ID']

In [24]:

merge_df = pd.merge(this_df, last_df[key], how='left', on=primary_key)

In [25]:
merge_df.head()

,ORIGIN_ID,CODE,CODE_NAME,TITLE,TEXT5_x,AUTHOR,CREATE_DATE_x,ORGAN_ID,CREATE_DATE_y,TEXT5_y
0,3644585,,中广核电力,2018年电力数据点评：2018年用电量同比增长8.5%，火电利用小时显著提升,NaN,"周妍,张爱宁",2019-01-30,2,2019-01-13,NaN
1,3646379,000001,平安银行,Q4银行持仓微降，仓位反弹可期,NaN,"沈娟,郭其伟",2019-01-30,77,2019-01-28,NaN
2,3645684,000001,平安银行,2月策略：政策与资金驱动，反弹行情或延续,NaN,廖志明,2019-01-30,2570,2019-01-25,NaN
3,3645685,000002,万科A,对当前板块表现的讨论（十二）：楼市的小阳春,NaN,"谢皓宇,卜文凯,白淑媛",2019-01-30,2,2019-01-22,NaN
4,3645207,000002,万科A,房地产销售周报：基本面依旧较弱、看好1月下旬后地产股机会,NaN,"陈天诚,徐超",2019-01-30,2570,2019-01-21,42.64


In [28]:
# merge 获取上一次最近的数据时， 若上一次数据为空，` 删除它们
# merge_df = merge_df.dropna(subset=['WRITE_DATE_y'])
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 4 to 356
Data columns (total 10 columns):
ORIGIN_ID        35 non-null int64
CODE             35 non-null object
CODE_NAME        35 non-null object
TITLE            35 non-null object
TEXT5_x          12 non-null float64
AUTHOR           35 non-null object
CREATE_DATE_x    35 non-null object
ORGAN_ID         35 non-null int64
CREATE_DATE_y    35 non-null datetime64[ns]
TEXT5_y          35 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 3.0+ KB


In [30]:
# merge_df

In [27]:
# 删除两次预测值均为空的数据
merge_df = merge_df.dropna(subset=['TEXT5_x','TEXT5_y'], how='all')

In [31]:
# cmb_report_research.ORGAN_ID=report_org_map.TGT_ORG_ID
sql = """SELECT distinct TGT_ORG_ID,TGT_ORG_NAME,TEXT1 FROM report_org_map"""
refer_df = pd.read_sql(sql, researchrpt_conn)

In [21]:
refer_df = refer_df.rename(columns = {"TGT_ORG_ID":"ORGAN_ID"})

In [22]:
final_df = pd.merge(merge_df, refer_df, how='left', on='ORGAN_ID') 

In [23]:
final_df = final_df.drop('ORGAN_ID', axis=1)

In [24]:
final_df.head()

,ORIGIN_ID,CODE,CODE_NAME,TITLE,TEXT5_x,AUTHOR,CREATE_DATE_x,CREATE_DATE_y,TEXT5_y,TGT_ORG_NAME,TEXT1
0,3644585,,中广核电力,2018年电力数据点评：2018年用电量同比增长8.5%，火电利用小时显著提升,NaN,"周妍,张爱宁",2019-01-30,2019-01-13,NaN,国泰君安,1786
1,3646379,000001,平安银行,Q4银行持仓微降，仓位反弹可期,NaN,"沈娟,郭其伟",2019-01-30,2019-01-28,NaN,华泰证券,26506
2,3645684,000001,平安银行,2月策略：政策与资金驱动，反弹行情或延续,NaN,廖志明,2019-01-30,2019-01-25,NaN,天风证券,15750
3,3645685,000002,万科A,对当前板块表现的讨论（十二）：楼市的小阳春,NaN,"谢皓宇,卜文凯,白淑媛",2019-01-30,2019-01-22,NaN,国泰君安,1786
4,3645207,000002,万科A,房地产销售周报：基本面依旧较弱、看好1月下旬后地产股机会,NaN,"陈天诚,徐超",2019-01-30,2019-01-21,42.64,天风证券,15750


### 获取current price 

In [5]:
# price_current
sql = """SELECT distinct ticker_symbol,trade_date,pre_close_price FROM mkt_equd where trade_date='%s'""" % this_date_str
refer_df = pd.read_sql(sql, datayesdb_conn)

In [9]:
# price_current
sql = """SELECT distinct ticker_symbol,trade_date,pre_close_price FROM mkt_equd where trade_date='%s'""" % '2019-05-29'
refer_df = pd.read_sql(sql, datayesdb_conn)

In [10]:
refer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3718 entries, 0 to 3717
Data columns (total 3 columns):
ticker_symbol      3718 non-null object
trade_date         3718 non-null object
pre_close_price    3718 non-null float64
dtypes: float64(1), object(2)
memory usage: 87.2+ KB


In [11]:
refer_df.head()

,ticker_symbol,trade_date,pre_close_price
0,000001,2019-05-29,12.49
1,000002,2019-05-29,27.62
2,000004,2019-05-29,23.44
3,000005,2019-05-29,3.08
4,000006,2019-05-29,5.65


In [12]:
refer_df.set_index("ticker_symbol").loc["00241"]

KeyError: u'the label [00241] is not in the [index]'

In [26]:
refer_df = refer_df.rename(columns = {"ticker_symbol":"CODE", "trade_date":"CREATE_DATE_x"})


In [27]:
final_df = pd.merge(final_df, refer_df, how='left', on=['CODE', 'CREATE_DATE_x']) 

In [28]:
# REPORT_ID
# SEC_CODE
# SEC_NAME
# TITLE
# REPORT_TYPE
# ORG_ID
# ORG_NAME
# AUTHOR
# THIS_WRITE_DATE
# LAST_WRITE_DATE
# THIS_TAR_PRICE
# LAST_TAR_PRICE
# CURRENT_PRICE
# TAR_PRICE_MARK


final_df.columns = ["REPORT_ID","SEC_CODE","SEC_NAME","TITLE","THIS_TAR_PRICE","AUTHOR","THIS_WRITE_DATE","LAST_WRITE_DATE","LAST_TAR_PRICE","ORG_NAME","ORG_ID","CURRENT_PRICE"]

In [29]:
# 最后添加两个不用进行表查询的列
final_df["REPORT_TYPE"] = "R03"

In [42]:

def compare(row):
    # 将本次评级与上次评级做对比，取常量。0未调（两次评级结果一样）；1上调（本次大于上次）；2下调（本次小于上次）；3未知

    x = row[0]
    y = row[1]
    try:
        x = row[0]
        y = row[1]
        if x is np.nan or x is None or y is None or y is np.nan:
            print("1")
            return 3
        x = float(x)
        y = float(y)
        if x > y:
            return 1
        elif x < y:
            return 2
        elif x == y:
            return 0
    except:
        print("2")
        return 3



In [34]:
final_df.head()

,REPORT_ID,SEC_CODE,SEC_NAME,TITLE,THIS_TAR_PRICE,AUTHOR,THIS_WRITE_DATE,LAST_WRITE_DATE,LAST_TAR_PRICE,ORG_NAME,ORG_ID,CURRENT_PRICE,REPORT_TYPE,TAR_PRICE_MARK
0,3644585,,中广核电力,2018年电力数据点评：2018年用电量同比增长8.5%，火电利用小时显著提升,NaN,"周妍,张爱宁",2019-01-30,2019-01-13,NaN,国泰君安,1786,NaN,R03,3
1,3646379,000001,平安银行,Q4银行持仓微降，仓位反弹可期,NaN,"沈娟,郭其伟",2019-01-30,2019-01-28,NaN,华泰证券,26506,11.00,R03,3
2,3645684,000001,平安银行,2月策略：政策与资金驱动，反弹行情或延续,NaN,廖志明,2019-01-30,2019-01-25,NaN,天风证券,15750,11.00,R03,3
3,3645685,000002,万科A,对当前板块表现的讨论（十二）：楼市的小阳春,NaN,"谢皓宇,卜文凯,白淑媛",2019-01-30,2019-01-22,NaN,国泰君安,1786,26.88,R03,3
4,3645207,000002,万科A,房地产销售周报：基本面依旧较弱、看好1月下旬后地产股机会,NaN,"陈天诚,徐超",2019-01-30,2019-01-21,42.64,天风证券,15750,26.88,R03,3


In [33]:
final_df["TAR_PRICE_MARK"] = final_df[["THIS_TAR_PRICE", "LAST_TAR_PRICE"]].apply(compare, axis=1)

In [32]:
final_df["THIS_TAR_PRICE"] = final_df["THIS_TAR_PRICE"].fillna(0).astype(int)
final_df["LAST_TAR_PRICE"] = final_df["LAST_TAR_PRICE"].fillna(0).astype(int)

In [33]:
final_df.head()

,REPORT_ID,SEC_CODE,SEC_NAME,TITLE,THIS_TAR_PRICE,AUTHOR,THIS_WRITE_DATE,LAST_WRITE_DATE,LAST_TAR_PRICE,ORG_NAME,ORG_ID,CURRENT_PRICE,REPORT_TYPE,TAR_PRICE_MARK
0,3644585,,中广核电力,2018年电力数据点评：2018年用电量同比增长8.5%，火电利用小时显著提升,0,"周妍,张爱宁",2019-01-30,2019-01-13,0,国泰君安,1786,NaN,R03,3
1,3646379,000001,平安银行,Q4银行持仓微降，仓位反弹可期,0,"沈娟,郭其伟",2019-01-30,2019-01-25,0,华泰证券,26506,11.00,R03,3
2,3645684,000001,平安银行,2月策略：政策与资金驱动，反弹行情或延续,0,廖志明,2019-01-30,2019-01-25,0,天风证券,15750,11.00,R03,3
3,3645685,000002,万科A,对当前板块表现的讨论（十二）：楼市的小阳春,0,"谢皓宇,卜文凯,白淑媛",2019-01-30,2019-01-22,0,国泰君安,1786,26.88,R03,3
4,3645207,000002,万科A,房地产销售周报：基本面依旧较弱、看好1月下旬后地产股机会,0,"陈天诚,徐超",2019-01-30,2019-01-13,0,天风证券,15750,26.88,R03,3


In [44]:
# refer_df

In [71]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228 entries, 0 to 227
Data columns (total 14 columns):
REPORT_ID          228 non-null int64
SEC_CODE           227 non-null object
SEC_NAME           228 non-null object
TITLE              228 non-null object
AUTHOR             228 non-null object
THIS_WRITE_DATE    228 non-null object
THIS_TAR_PRICE     81 non-null object
ORG_ID             228 non-null int64
ORG_NAME           228 non-null object
LAST_WRITE_DATE    110 non-null object
LAST_TAR_PRICE     36 non-null object
CURRENT_PRICE      178 non-null float64
REPORT_TYPE        228 non-null object
TAR_PRICE_MARK     228 non-null int64
dtypes: float64(1), int64(3), object(10)
memory usage: 26.7+ KB


# 目标价调整表

# R04

In [36]:
import numpy as np

In [32]:
# base_df
filter_str = "WRITE_DATE" # 根据 num 列分开1，2 后， 需要从num=1的数据中筛选出日期为查询日期的数据
this_date_str = '2019-03-21'
primary_key = ["SEC_CODE", "ORG_ID"] # 用于merge
last_key = ["WRITE_DATE", "tar_price"] # 同一个机构上次对本证券做的预测报告的撰写日期 同一个机构上次对本证券做的预测报告的评级

In [33]:
sql = """
SELECT ID, SEC_CODE, SEC_NAME, TITLE, AUTHOR, WRITE_DATE, tar_price, ORG_ID, ORG_NAME, num
FROM (
	SELECT ID, SEC_CODE, SEC_NAME, TITLE, AUTHOR, WRITE_DATE, tar_price, ORG_ID, ORG_NAME,
		   (@row_number:=
			   case when SEC_CODE = @pre_code and ORG_ID = @pre_organ_id
			then @row_number+1
			 else 1
			end
		   ) as num,
		(@pre_code := SEC_CODE),
		(@pre_organ_id := ORG_ID)
	FROM rr_main,(select @row_number:=1,@pre_code:='',@pre_organ_id:='') a
	where WRITE_DATE <= '%s' and QA_ACTIVE_FLG=1 and QA_MANUAL_FLG=1 and  REPORT_TYPE_N='R04'
	order by SEC_CODE,ORG_ID,WRITE_DATE desc
) a
where a.num<=2
""" % (this_date_str)
base_df = pd.read_sql(sql, researchrpt_conn)

In [4]:
tuple=tuple([2,3,4])

In [5]:
print tuple

(2, 3, 4)


In [6]:
a={"a":1}

In [7]:
a["b"]

KeyError: 'b'

In [34]:
rr_main.id = 

In [38]:
base_df
# base_df["WRITE_DATE"] = base_df["WRITE_DATE"].astype(str)
# base_df.info()
# base_df[base_df[filter_str]=='2018-11-30']
# base_df[base_df['WRITE_DATE']=='2018-11-30']
base_df = base_df.fillna(value=np.nan)

In [39]:
# 用num的值区分次序
last_df = base_df[base_df["num"]==2]
last_df = last_df.drop("num", 1)

In [40]:
# last_df

In [41]:
base_df[filter_str] = base_df[filter_str].astype(str)
this_df = base_df[(base_df["num"]==1) & (base_df[filter_str]==this_date_str)]
this_df = this_df.drop("num", 1)


In [42]:
# this_df

In [43]:
if len(this_df) == 0:
    # 说明没有目标数据， 可以返回
    pass


In [44]:
# primary_key.append(last_key)
# primary_key
# last_key
# last_df[[last_key.extend(primary_key)]]

In [45]:
key = last_key + primary_key


In [46]:

merge_df = pd.merge(this_df, last_df[key], how='left', on=primary_key)

In [47]:
# merge 获取上一次最近的数据时， 若上一次数据为空， 删除它们
# merge_df = merge_df.dropna(subset=['WRITE_DATE_y'])

In [48]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 229 entries, 0 to 228
Data columns (total 11 columns):
ID              229 non-null int64
SEC_CODE        228 non-null object
SEC_NAME        229 non-null object
TITLE           229 non-null object
AUTHOR          229 non-null object
WRITE_DATE_x    229 non-null object
tar_price_x     81 non-null object
ORG_ID          229 non-null int64
ORG_NAME        229 non-null object
WRITE_DATE_y    111 non-null object
tar_price_y     36 non-null object
dtypes: int64(2), object(9)
memory usage: 21.5+ KB


In [51]:
merge_df.head()

,ID,SEC_CODE,SEC_NAME,TITLE,AUTHOR,WRITE_DATE_x,tar_price_x,ORG_ID,ORG_NAME,WRITE_DATE_y,tar_price_y
3,3683064,000100,TCL集团,TCL集团（000100）：资产重组持续推进，业务架构清晰启航,"陈子仪,朱默辰,李阳",2019-03-21,4.16,1343,海通证券,NaN,NaN
5,3683101,000401,冀东水泥,冀东水泥（000401）：京津冀龙头，受益于区域重点工程提速,"李可悦,陈彦",2019-03-21,21.39,26455,中金公司,2018-04-19,16.50
7,3682449,000401,冀东水泥,冀东水泥（000401）：Q4经营略超预期，19年表现将更为靓丽,"黄诗涛,石峰源",2019-03-21,NaN,27276,国盛证券,2019-01-24,18.5
11,3683385,000639,西王食品,西王食品（000639）：健康食品旗舰，边际改善再出发,王克宇,2019-03-21,13.97,26465,东方证券,NaN,NaN
12,3683151,000723,美锦能源,美锦能源（000723）：立足传统焦化业务，全面布局氢能源产业,"彭鑫,杨藻",2019-03-21,8.4,15750,天风证券,NaN,NaN


In [50]:
# merge_df.to_csv("write_date_2019-03-21.csv", index=False, encoding='utf-8')
merge_df = merge_df.dropna(subset=['tar_price_x', 'tar_price_y'], how='all')


In [54]:
# price_current
sql = """SELECT distinct ticker_symbol,trade_date,pre_close_price FROM mkt_equd where trade_date='%s'""" % this_date_str
refer_df = pd.read_sql(sql, datayesdb_conn)

In [55]:
refer_df = refer_df.rename(columns = {"ticker_symbol":"SEC_CODE", "trade_date":"WRITE_DATE_x"})


In [65]:
# final_df.head()
final_df = merge_df
final_df = pd.merge(final_df, refer_df, how='left', on=['SEC_CODE', 'WRITE_DATE_x']) 

In [70]:
final_df.head()

,REPORT_ID,SEC_CODE,SEC_NAME,TITLE,AUTHOR,THIS_WRITE_DATE,THIS_TAR_PRICE,ORG_ID,ORG_NAME,LAST_WRITE_DATE,LAST_TAR_PRICE,CURRENT_PRICE,REPORT_TYPE,TAR_PRICE_MARK
0,3686849,None,腾讯音乐,腾讯音乐：内容护城河高筑+付费率上升，在线音乐业务增长强劲,张忆东,2019-03-21,None,26548,兴业证券,NaN,NaN,NaN,R04,2
1,3682728,000028,国药一致,国药一致（000028）18年业绩快报点评：业绩拐点已现，最大药店网络将迎价值重估,"林小伟,梁东旭",2019-03-21,None,26504,光大证券,NaN,NaN,49.10,R04,2
2,3687394,00003,香港中华煤气,香港中华煤气（00003）：2018年年度业绩超出市场预期，维持“买入”评级,郭勇,2019-03-21,None,1786,国泰君安证券,2018-08-23,None,NaN,R04,0
3,3683064,000100,TCL集团,TCL集团（000100）：资产重组持续推进，业务架构清晰启航,"陈子仪,朱默辰,李阳",2019-03-21,4.16,1343,海通证券,NaN,NaN,3.82,R04,1
4,3682572,000401,冀东水泥,冀东水泥（000401）：京津冀水泥龙头，量价弹性持续释放,"盛昌盛,武浩翔",2019-03-21,None,15750,天风证券,2019-01-11,None,17.05,R04,0


In [66]:
# # rename columns 
# REPORT_ID
# SEC_CODE
# SEC_NAME
# TITLE
# REPORT_TYPE
# ORG_ID
# ORG_NAME
# AUTHOR
# THIS_WRITE_DATE
# LAST_WRITE_DATE

final_df.columns = ["REPORT_ID","SEC_CODE","SEC_NAME","TITLE","AUTHOR","THIS_WRITE_DATE","THIS_TAR_PRICE","ORG_ID","ORG_NAME","LAST_WRITE_DATE","LAST_TAR_PRICE","CURRENT_PRICE"]

In [67]:
# 最后添加两个不用进行表查询的列
final_df["REPORT_TYPE"] = "R04"

In [68]:
# 将本次评级与上次评级做对比，取常量。0未调（两次评级结果一样）；1上调（本次大于上次）；2下调（本次小于上次）；3未知

final_df["TAR_PRICE_MARK"] = final_df[["THIS_TAR_PRICE", "LAST_TAR_PRICE"]].apply(compare, axis=1)

In [69]:
# final_df["THIS_TAR_PRICE"] = final_df["THIS_TAR_PRICE"].fillna(0).astype(int)
# final_df["LAST_TAR_PRICE"] = final_df["LAST_TAR_PRICE"].fillna(0).astype(int)

In [72]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228 entries, 0 to 227
Data columns (total 14 columns):
REPORT_ID          228 non-null int64
SEC_CODE           227 non-null object
SEC_NAME           228 non-null object
TITLE              228 non-null object
AUTHOR             228 non-null object
THIS_WRITE_DATE    228 non-null object
THIS_TAR_PRICE     81 non-null object
ORG_ID             228 non-null int64
ORG_NAME           228 non-null object
LAST_WRITE_DATE    110 non-null object
LAST_TAR_PRICE     36 non-null object
CURRENT_PRICE      178 non-null float64
REPORT_TYPE        228 non-null object
TAR_PRICE_MARK     228 non-null int64
dtypes: float64(1), int64(3), object(10)
memory usage: 26.7+ KB


# 当底层数据有修改时

### 如id=1的记录tar_price值改变， 需要按seccode把所有日期前后的记录都重算一遍


In [46]:
id=3504779
pd.datetime.now()

datetime.datetime(2019, 5, 25, 14, 14, 49, 137000)

In [5]:
sql="""SELECT ID, SEC_CODE, SEC_NAME, TITLE, AUTHOR, WRITE_DATE, tar_price, ORG_ID, ORG_NAME
        from rr_main
        where sec_code=000001 and ORG_ID=15750 order by write_date desc

 """
base_df = pd.read_sql(sql, researchrpt_conn)

c:\python27\lib\site-packages\pymysql\cursors.py:166: Warning: (1292, u"Truncated incorrect DOUBLE value: 'X15336'")
  result = self._query(query)


In [6]:
base_df 

,ID,SEC_CODE,SEC_NAME,TITLE,AUTHOR,WRITE_DATE,tar_price,ORG_ID,ORG_NAME
0,3716132,000001,平安银行,平安银行（000001）：业绩步入兑现期，零售银行新标杆,廖志明,2019-04-24,17.21,15750,天风证券
1,3668837,000001,平安银行,平安银行（000001）：资产质量有望持续向好，业绩弹性在增强,"廖志明,林瑾璐",2019-03-07,17.32,15750,天风证券
2,3653147,000001,平安银行,平安银行（000001）：Fintech+流量，零售转型龙头,"廖志明,林瑾璐",2019-02-18,15.85,15750,天风证券
3,3507060,000001,平安银行,平安银行（000001）：不良偏离度改善超预期，ROE提升可期,"廖志明,林瑾璐",2019-01-04,15.85,15750,天风证券
4,3503804,000001,平安银行,平安银行（000001）：可转债发行获批，零售转型潜力大,"廖志明,林瑾璐",2018-12-27,15.34,15750,天风证券
5,2235492,000001,平安银行,平安银行（000001）：3Q18业绩：营收大幅改善，不良仍在出清中,廖志明,2018-10-24,16.63,15750,天风证券
6,2211416,000001,平安银行,平安银行（000001）：业绩弹性增强，市场利率大降显著利好,"廖志明,林瑾璐",2018-08-16,16.65,15750,天风证券
7,2156098,000001,平安银行,"平安银行(000001)：零售转型显成效,业绩弹性明显提升","廖志明,林瑾璐",2018-04-20,19.0000,15750,天风证券
8,2140047,000001,平安银行,"平安银行(000001)：盈利符合预期,零售转型成效显著","廖志明,林瑾璐",2018-03-15,19.0000,15750,天风证券
9,2086035,000001,平安银行,"平安银行(000001)：零售转型有基因、有方向、有资源,坚定看好","廖志明,林瑾璐",2017-11-20,19.0000,15750,天风证券


In [26]:
# shift
shift_df = base_df[["SEC_CODE","ORG_ID","WRITE_DATE","tar_price"]]
last_df = shift_df.shift(-1)[["WRITE_DATE","tar_price"]]
last_df.columns=['write_date_y', 'tar_price_y']
final_df = pd.concat([base_df,last_df], axis=1)

In [45]:
final_df.head(1)

,REPORT_ID,SEC_CODE,SEC_NAME,TITLE,AUTHOR,THIS_WRITE_DATE,THIS_TAR_PRICE,ORG_ID,ORG_NAME,LAST_WRITE_DATE,LAST_TAR_PRICE,REPORT_TYPE,TAR_PRICE_MARK
0,3716132,000001,平安银行,平安银行（000001）：业绩步入兑现期，零售银行新标杆,廖志明,2019-04-24,17.21,15750,天风证券,2019-03-07,17.32,R04,2.0


In [28]:
# # rename columns 
# REPORT_ID
# SEC_CODE
# SEC_NAME
# TITLE
# REPORT_TYPE
# ORG_ID
# ORG_NAME
# AUTHOR
# THIS_WRITE_DATE
# LAST_WRITE_DATE

final_df.columns = ["REPORT_ID","SEC_CODE","SEC_NAME","TITLE","AUTHOR","THIS_WRITE_DATE","THIS_TAR_PRICE","ORG_ID","ORG_NAME","LAST_WRITE_DATE","LAST_TAR_PRICE"]

In [29]:
# 最后添加两个不用进行表查询的列
final_df["REPORT_TYPE"] = "R04"

In [43]:
# 将本次评级与上次评级做对比，取常量。0未调（两次评级结果一样）；1上调（本次大于上次）；2下调（本次小于上次）；3未知

final_df["TAR_PRICE_MARK"] = final_df[["THIS_TAR_PRICE", "LAST_TAR_PRICE"]].apply(compare, axis=1)